In [3]:
import numpy as np
import pandas as pd 


In [4]:
#Load the dataset 

df = pd.read_csv("..\Python_Checkpoints\STEG_BILLING_HISTORY.csv")

C:\Users\zakaa\AppData\Local\Temp\ipykernel_44148\2774413690.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("..\Python_Checkpoints\STEG_BILLING_HISTORY.csv")


In [5]:
#Display the ten first lines

client_0_bills =pd.DataFrame(df.head(10))
client_0_bills

,client_id,invoice_date,tarif_type,counter_number,counter_statue,counter_code,reading_remarque,counter_coefficient,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,old_index,new_index,months_number,counter_type
0,train_Client_0,2014-03-24,11,1335667.0,0,203,8.0,1,82,0,0,0,14302,14384,4,ELEC
1,train_Client_0,2013-03-29,11,1335667.0,0,203,6.0,1,1200,184,0,0,12294,13678,4,ELEC
2,train_Client_0,2015-03-23,11,1335667.0,0,203,8.0,1,123,0,0,0,14624,14747,4,ELEC
3,train_Client_0,2015-07-13,11,1335667.0,0,207,8.0,1,102,0,0,0,14747,14849,4,ELEC
4,train_Client_0,2016-11-17,11,1335667.0,0,207,9.0,1,572,0,0,0,15066,15638,12,ELEC
5,train_Client_0,2017-07-17,11,1335667.0,0,207,9.0,1,314,0,0,0,15638,15952,8,ELEC
6,train_Client_0,2018-12-07,11,1335667.0,0,207,9.0,1,541,0,0,0,15952,16493,12,ELEC
7,train_Client_0,2019-03-19,11,1335667.0,0,207,9.0,1,585,0,0,0,16493,17078,8,ELEC
8,train_Client_0,2011-07-22,11,1335667.0,0,203,9.0,1,1200,186,0,0,7770,9156,4,ELEC
9,train_Client_0,2011-11-22,11,1335667.0,0,203,6.0,1,1082,0,0,0,9156,10238,4,ELEC


In [20]:
#Data type of the 'client_0_bills'

client_0_bills.dtypes

client_id                object
invoice_date             object
tarif_type                int64
counter_number          float64
counter_statue           object
counter_code              int64
reading_remarque        float64
counter_coefficient       int64
consommation_level_1      int64
consommation_level_2      int64
consommation_level_3      int64
consommation_level_4      int64
old_index                 int64
new_index                 int64
months_number             int64
counter_type             object
dtype: object

In [25]:
client_0_bills.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   client_id             10 non-null     object 
 1   invoice_date          10 non-null     object 
 2   tarif_type            10 non-null     int64  
 3   counter_number        10 non-null     float64
 4   counter_statue        10 non-null     object 
 5   counter_code          10 non-null     int64  
 6   reading_remarque      10 non-null     float64
 7   counter_coefficient   10 non-null     int64  
 8   consommation_level_1  10 non-null     int64  
 9   consommation_level_2  10 non-null     int64  
 10  consommation_level_3  10 non-null     int64  
 11  consommation_level_4  10 non-null     int64  
 12  old_index             10 non-null     int64  
 13  new_index             10 non-null     int64  
 14  months_number         10 non-null     int64  
 15  counter_type          10 n

In [7]:
#Rows and columns

rows, columns = df.shape
print(
    f"The dataset has {rows} rows and {columns} columns."
)

The dataset has 4476749 rows and 16 columns.


In [ ]:
# categorical features
client_0_bills.nunique()

client_id                1
invoice_date            10
tarif_type               1
counter_number           1
counter_statue           1
counter_code             2
reading_remarque         3
counter_coefficient      1
consommation_level_1     9
consommation_level_2     3
consommation_level_3     1
consommation_level_4     1
old_index               10
new_index               10
months_number            3
counter_type             1
dtype: int64

In [24]:
#Memory usage
print(f"The memory consumation is of: {df.memory_usage(deep=True).sum() / (1024 ** 2):.2f} MB")   # in MB



The memory consumation is of: 1430.36 MB


In [36]:
#Inspect the dataset for potential missing values

df.isnull().sum()

client_id                  0
invoice_date               0
tarif_type                 0
counter_number            48
counter_statue             0
counter_code               0
reading_remarque        4531
counter_coefficient        0
consommation_level_1       0
consommation_level_2       0
consommation_level_3       0
consommation_level_4       0
old_index                  0
new_index                  0
months_number              0
counter_type               0
dtype: int64

In [ ]:
#Strategy on handling missing Values

#Checking Forward and Backward Fill 
missing_rows = df.loc[df[['counter_number', 'reading_remarque']].isna().any(axis=1), ['counter_number', 'reading_remarque']]
missing_rows_indices = missing_rows.index
context_indices = set()

for idx in missing_rows_indices:
    context_indices.update([idx - 1, idx, idx + 1])

context_df = df.loc[df.index.intersection(context_indices),['counter_number', 'reading_remarque']].sort_index()
context_df 
#It seems this Strategy might work, it follows a pattern with specific condition.


""" 
    Reading from a specific meter under a specific contract.
        - client_id tells you who
        - tarif_type and counter_type tell you what kind of service
        - counter_number tells you which device
    So our imputation logic should say:
    “Fill missing meter IDs only when we are confident it’s the same meter under the same conditions.”

 """
data_copy = df.copy()
group_cols = ['client_id', 'tarif_type', 'counter_type']
data_copy['counter_number'] = data_copy.groupby(group_cols)['counter_number'].transform(lambda x: x.ffill().bfill())

data_copy['reading_remarque'] = data_copy['reading_remarque'].fillna('unknown')

data_copy.isnull().sum().sum()


np.int64(0)

In [38]:
data_copy.head()

,client_id,invoice_date,tarif_type,counter_number,counter_statue,counter_code,reading_remarque,counter_coefficient,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,old_index,new_index,months_number,counter_type
0,train_Client_0,2014-03-24,11,1335667.0,0,203,8.0,1,82,0,0,0,14302,14384,4,ELEC
1,train_Client_0,2013-03-29,11,1335667.0,0,203,6.0,1,1200,184,0,0,12294,13678,4,ELEC
2,train_Client_0,2015-03-23,11,1335667.0,0,203,8.0,1,123,0,0,0,14624,14747,4,ELEC
3,train_Client_0,2015-07-13,11,1335667.0,0,207,8.0,1,102,0,0,0,14747,14849,4,ELEC
4,train_Client_0,2016-11-17,11,1335667.0,0,207,9.0,1,572,0,0,0,15066,15638,12,ELEC


In [37]:
#Run a descriptive analysis on numeric features (columns)
data_copy.describe().T

,count,mean,std,min,25%,50%,75%,max
tarif_type,4476749.0,2.012804e+01,1.347256e+01,8.0,11.0,11.0,40.0,4.500000e+01
counter_number,4476749.0,1.230587e+11,1.657267e+12,0.0,121108.0,494561.0,1115161.0,2.798115e+13
counter_code,4476749.0,1.724884e+02,1.338871e+02,0.0,5.0,203.0,207.0,6.000000e+02
counter_coefficient,4476749.0,1.003040e+00,3.083466e-01,0.0,1.0,1.0,1.0,5.000000e+01
consommation_level_1,4476749.0,4.109795e+02,7.573080e+02,0.0,79.0,274.0,600.0,9.999100e+05
consommation_level_2,4476749.0,1.093225e+02,1.220123e+03,0.0,0.0,0.0,0.0,9.990730e+05
consommation_level_3,4476749.0,2.030620e+01,1.574239e+02,0.0,0.0,0.0,0.0,6.449200e+04
consommation_level_4,4476749.0,5.292588e+01,8.754725e+02,0.0,0.0,0.0,0.0,5.479460e+05
old_index,4476749.0,1.776700e+04,4.036693e+04,0.0,1791.0,7690.0,21660.0,2.800280e+06
new_index,4476749.0,1.834970e+04,4.095321e+04,0.0,2056.0,8192.0,22343.0,2.870972e+06
